In [11]:
import util_funs.globalsettings as gs
from langchain.agents import create_csv_agent
from langchain.llms import OpenAI

from langchain.vectorstores.chroma import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain.schema import HumanMessage, AIMessage

from langchain.agents import AgentType, initialize_agent

from langchain import LLMMathChain, SerpAPIWrapper
from langchain.tools import BaseTool, StructuredTool, Tool, tool

from langchain.memory import ConversationBufferMemory

from langchain import PromptTemplate

In [3]:
model = ChatOpenAI(
        model_name="gpt-3.5-turbo",
        temperature="0",
        # verbose=True
    )

In [7]:
agent = create_csv_agent(model, gs.the_files.CSV_FILES[0], verbose=True)

In [10]:
vector_store_pdf = Chroma(
    collection_name='pdf_files',
    embedding_function=OpenAIEmbeddings(),
    persist_directory=gs.the_folders.DIR_PERSIST
    )
pdf_retriever = vector_store_pdf.as_retriever()

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

qa_chain = ConversationalRetrievalChain.from_llm(
    model,
    retriever=pdf_retriever,
    return_source_documents=True,
    memory =  memory
    # verbose=True,
    )

Using embedded DuckDB with persistence: data will be stored in: C:\Users\migue\Documents\PROYECTOS DATA SCIENCE\langchain\PYTHON\util_funs\..\..\data\chroma


In [34]:
if_prompt = PromptTemplate(
    input_variables=["query"], 
    template="Tu trabajo es clasificar la pregunta siguiente pregunta. \
              PREGUNTA: {query}  \
              Estas son tus INSTRUCCIONES para clasificar la pregunta:\
              - Si tiene que ver con el paro / desempleo en España (por ejemplo: 'Cuál fue  \
                la tasa de paro total en España en 2023T1?'), contestarás \
                con la palabra: 'csv_spain_retriever'  \
              - Si tiene que ver con el paro / desempleo en la Unión Europea (UE)  \
                (por ejemplo, 'cual fue la tasa de desempleo en la UE en 2021?'),  \
                contestarás con la palabra 'csv_ue_retriever'  \
              - Si no es ninguno de los casos anteriores o no sabes la respuessta contestarás \
                con la palabra 'csv_pdf_retriever'"           
)

In [24]:
if_prompt.format(query = "Cuál fue la tasa de paro total en España en 2023T1?")

"Tu trabajo es clasificar la pregunta siguiente pregunta.               PREGUNTA: Cuál fue la tasa de paro total en España en 2023T1?                Estas son tus INSTRUCCIONES para clasificar la pregunta:              - Si tiene que ver con el paro / desempleo en España (por ejemplo: 'Cuál fue                  la tasa de paro total en España en 2023T1?'), contestarás 'csv_spain_retriever'                - Si tiene que ver con el paro / desempleo en la Unión Europea (UE)                  (por ejemplo, 'cual fue la tasa de desempleo en la UE en 2021?'),                  contestarás 'csv_ue_retriever'                - Si no es ninguno de los casos anteriores contestarás 'csv_pdf_retriever'"

In [35]:
from langchain.llms import OpenAI
llm = OpenAI(model_name="text-ada-001")
llm(if_prompt.format(query = "Cuál fue la tasa de paro total en España en 2023T1?"))

".'"